# 07. PyTorch Experiments Tracking

## Different ways to track experiments
* Python dictionaries, CSV files, print outs
* TensorBoard - https://www.tensorflow.org/tensorboard
* Weights & Biases - https://wandb.ai/site/experiment-tracking
* MLflow - https://mlflow.org/

## What is going to be covered in this module:
* Getting setup (importing previously written code)
* Introduce experiment tracking with PyTorch
* Building several modelling experiments for FoodVision Mini
* Evaluating modelling experiments with TensorBoard
* Making predictions with the best performing model on custom data

In [1]:
import torch
import torchvision

print(torch.__version__)
print(torchvision.__version__)

2.9.1+cu126
0.24.1+cu126


In [ ]:
# Continue with regular imports
import matplotlib.pyplot as plt

from torch import nn
from torchvision import transforms

# Try to get torchinfo, install it if it doesn't work
try:
    from torchinfo import summary
except:
    print("[INFO] could not find torchinfo... installing it.")
    !pip install -q torchinfo
    from torchinfo import summary
    
# Try to import the going_modular directory, download it from GitHub if it doesn't work
try:
    from going_modular.going_modular import data_setup, engine
except:
    # Get the going_modular scripts
    print("[INFO] Couldn't find going_modular scripts... downloading them from GitHub.")
    !git clone https://github.com/mrdbourke/pytorch-deep-learning
    !mv pytorch-deep-learning/going_modular .
    !rm -rf pytorch-deep-learning
    from going_modular.going_modular import data_setup, engine

[INFO] Couldn't find going_modular scripts... downloading them from GitHub.


In [ ]:
# Setup device agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"
device

In [ ]:
# Set seeds
def set_seeds(seed: int=42):
    """
    Sets random sets for torch operations.
    
    Args:
        seed (int, optional): Random seed to set. Defaults to 42.
    """
    # Set the seed for general torch operations
    torch.manual_seed(seed)
    # Set the seed for CUDA torch operations (ones that happen on the GPU)
    torch.cuda.manual_seed(seed)